In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import random
import copy

######## perturbate tools ########

def round_shuffler(original_qa: dict):
    """return:
        shuffled QAs
        round map: {new_dialogue_key: original_dialogue_key} 
    """
    original_round_keys = list(original_qa.keys())
    shuffled_round_keys = random.sample(original_round_keys,len(original_round_keys))
    new_dialogue = {}
    new_dialogue_round_mapping = {}
    for count, round_key in enumerate(shuffled_round_keys):
        new_round_key = "Round {}".format(count+1)
        new_dialogue_round_mapping[new_round_key] = round_key
        new_dialogue[new_round_key]=original_qa[round_key]
    return new_dialogue, new_dialogue_round_mapping


def round_reducer(original_qa: dict, drop_round_keys=[], drop_ratio=0.3):
    """return:
        round-reduced QAs
        round map: {new_dialogue_key: original_dialogue_key} 
        list of reduced original rounds: ["Round x",...]
    """
    original_round_keys = list(original_qa.keys())
    new_dialogue = {}
    new_dialogue_round_mapping = {}

    # specified round to be moved:
    def remove_round_with_inputkeys():
        remain_original_round_keys = [item for item in original_round_keys if item not in drop_round_keys]
        for i,original_round_key in enumerate(remain_original_round_keys):
            new_thisround_key = "Round {}".format(i+1)
            new_thisround_qa = original_qa[original_round_key]
            new_dialogue[new_thisround_key] = new_thisround_qa
            new_dialogue_round_mapping[new_thisround_key] = original_round_key
        return new_dialogue, new_dialogue_round_mapping, drop_round_keys
    
    if drop_round_keys:
        # specify round keys to remove
        return remove_round_with_inputkeys()
    else:
        # randomly choose drop_ratio(30% default) keys to removed
        if drop_ratio<1:
            drop_round_keys=random.sample(original_round_keys,int(len(original_round_keys)*drop_ratio))
        elif drop_ratio<len(original_round_keys):
            drop_round_keys=random.sample(original_round_keys,round(drop_ratio))
        else:
            raise(ValueError("Bad drop_ratio: drop_ratio should less then len(original_round_keys)"))
        return remove_round_with_inputkeys()


def round_duplicator(original_qa: dict, duplicate_original_round_keys=[], insert_to_new_dialogue_round_keys=[], duplicate_ratio=0.2):
    """return:
        round-duplicated QAs
        round map: {new_dialogue_key:original_dialogue_key} 
        duplicated original rounds: new_dialogue_key -> original_dialogue_key, original_dialogue_key exist more than once
    """
    original_round_keys = list(original_qa.keys())
    new_dialogue = {}
    new_dialogue_round_mapping = {}

    new_dialogue_duplicated_from_original = {}

    # error handling
    if len(duplicate_original_round_keys) != len(insert_to_new_dialogue_round_keys): raise(SyntaxError("Length not match: duplicate_original_round_keys, insert_to_new_dialogue_round_keys"))
    for round_key in duplicate_original_round_keys:
        if round_key not in original_round_keys: raise(KeyError("Bad key in duplicate_original_round_keys: {}".format(round_key)))

    def duplicate_round_with_keys():
        original_round_keys_reverse = original_round_keys[::-1]
        for new_round_key in new_dialogue_round_keys: 
            # if this round is a specified place to insert duplicated round
            if new_round_key in insert_to_new_dialogue_round_keys: 
                thisnewround_use_original_round_key = duplicate_original_round_keys[insert_to_new_dialogue_round_keys.index(new_round_key)]
                new_dialogue[new_round_key] = original_qa[thisnewround_use_original_round_key]
                new_dialogue_round_mapping[new_round_key] = thisnewround_use_original_round_key
                new_dialogue_duplicated_from_original[new_round_key] = thisnewround_use_original_round_key
            # follow original sequence
            else: 
                thisnewround_use_original_round_key = original_round_keys_reverse.pop()
                new_dialogue[new_round_key] = original_qa[thisnewround_use_original_round_key]
                new_dialogue_round_mapping[new_round_key] = thisnewround_use_original_round_key
        return new_dialogue, new_dialogue_round_mapping, new_dialogue_duplicated_from_original

    # if specified round to duplicate and round number to place:
    if duplicate_original_round_keys:
        new_dialogue_round_keys = original_round_keys+["Round {}".format(len(original_round_keys)+new_round+1) for new_round in range(len(duplicate_original_round_keys))]
        return duplicate_round_with_keys()
    
    # randomly choose rounds to duplicate and places to insert
    else:
        if duplicate_ratio<1:
            duplicate_original_round_keys = random.sample(original_round_keys, int(len(original_round_keys)*duplicate_ratio))
            new_dialogue_round_keys = original_round_keys+["Round {}".format(len(original_round_keys)+new_round+1) for new_round in range(len(duplicate_original_round_keys))]
            insert_to_new_dialogue_round_keys = random.sample(new_dialogue_round_keys, int(len(original_round_keys)*duplicate_ratio))
            return duplicate_round_with_keys()
        elif duplicate_ratio<len(original_round_keys):
            duplicate_original_round_keys = random.sample(original_round_keys, round(duplicate_ratio))
            new_dialogue_round_keys = original_round_keys+["Round {}".format(len(original_round_keys)+new_round+1) for new_round in range(len(duplicate_original_round_keys))]
            insert_to_new_dialogue_round_keys = random.sample(new_dialogue_round_keys, round(duplicate_ratio))
            return duplicate_round_with_keys()
        else:
            raise(ValueError("Bad duplicate_ratio: duplicate_ratio should less then len(original_round_keys)"))


def round_reduce_shuffle(original_qa: dict, drop_round_keys=[], drop_ratio=0.3):
    """return:
        round-reduced-shuffled QAs
        round map: {new_dialogue_key: original_dialogue_key} 
        list of reduced original rounds: ["Round x",...]
    """
    reduced_dialogue, reduced_dialogue_round_mapping, dropped_round_keys = round_reducer(original_qa, drop_round_keys, drop_ratio)
    shuffled_reduced_dialogue, shuffled_reduced_dialogue_round_mapping = round_shuffler(reduced_dialogue)
    to_original_mapping={}
    for round_key in list(shuffled_reduced_dialogue.keys()):
        to_original_mapping[round_key] = reduced_dialogue_round_mapping[shuffled_reduced_dialogue_round_mapping[round_key]]
    return shuffled_reduced_dialogue, to_original_mapping, dropped_round_keys

def round_shuffle_duplicate(original_qa: dict, duplicate_original_round_keys=[], insert_to_new_dialogue_round_keys=[], duplicate_ratio=0.2):
    """return:
        round-duplicated-shuffled QAs
        round map: {new_dialogue_key: original_dialogue_key}
        duplicated original rounds: {new_dialogue_key: original_dialogue_key}, original_dialogue_key exist more than once. might not that useful
    """
    shuffled_dialogue, shuffled_dialogue_round_mapping = round_shuffler(original_qa)
    shuffled_duplicated_dialogue, shuffled_duplicated_dialogue_round_mapping, shuffled_duplicated_dialogue_duplicated_from_original = round_duplicator(shuffled_dialogue, duplicate_original_round_keys, insert_to_new_dialogue_round_keys, duplicate_ratio)
    
    to_original_mapping={}
    to_original_info = {}

    for round_key in list(shuffled_duplicated_dialogue.keys()):
        to_original_mapping[round_key] = shuffled_dialogue_round_mapping[shuffled_duplicated_dialogue_round_mapping[round_key]]

    for round_key in list(shuffled_duplicated_dialogue_duplicated_from_original.keys()):
        to_original_info[round_key] = shuffled_dialogue_round_mapping[shuffled_duplicated_dialogue_round_mapping[round_key]]
        
    return shuffled_duplicated_dialogue, to_original_mapping, shuffled_duplicated_dialogue_duplicated_from_original, to_original_info

######## verify tools ########

wh_words=["who", "what", "where", "when", "why", "which", "whose", "whom", "how"]

aux_words={"be": ["am", "is", "are", "was", "were"],
           "have": ["has", "have", "had"],
           "do": ["do", "does", "did"],
           "modal Verbs":["can", "could", "will", "would", "shall", "should", "may", "might"]}

pronouns={
    "Personal Pronouns":["I", "you", "he", "she", "it", "we", "they", "me", "him", "her", "us", "them"],
    "Possessive Pronouns":["mine", "yours", "his", "hers", "ours", "theirs"],
    "Possessive Adjectives":["my", "your", "his", "her", "its", "our", "their"],
    "Reflexive Pronouns":["myself", "yourself", "himself", "herself", "itself", "ourselves", "themselves"],
    "Demonstrative Pronouns":["this", "that", "these", "those"], # coreference models can not handle demonstrative pronouns
}

regular_pronouns={
    "Personal Pronouns":["he", "she", "it", "they", "him", "her", "them"],
    "Possessive Pronouns":["yours", "his", "hers", "theirs"],
    "Possessive Adjectives":["your", "his", "her", "its", "their"],
    "Reflexive Pronouns":["himself", "herself", "itself", "themselves"],
}

DT_pronoun = {
    "Demonstrative Pronouns":["this", "that", "these", "those"],
    "Indefinite Pronouns":["all", "another", "any", "anybody", "anyone", "both", "each", "either", "everyone", "few", "many", "none", "one", "some", "somebody", "several"],
 }


all_aux_words = []
for key in aux_words.keys():
    for item in aux_words[key]:
        if item not in all_aux_words:
            all_aux_words.append(item)

all_pronouns = []
for key in pronouns.keys():
    for item in pronouns[key]:
        if item not in all_pronouns:
            all_pronouns.append(item)

all_regular_pronouns = []
for key in regular_pronouns.keys():
    for item in regular_pronouns[key]:
        if item not in all_regular_pronouns:
            all_regular_pronouns.append(item)

import spacy
import coreferee
nlp = spacy.load("en_core_web_trf")
nlp.add_pipe('coreferee')

def flatten(lst):
    flat_list = []
    for item in lst:
        if isinstance(item, list):
            flat_list.extend(flatten(item))  # Recursively flatten the list
        else:
            flat_list.append(item)
    return flat_list

def self_resolvable(input_text, PRINT_FLAG=False):
    """
    input: input_text, PRINT_FLAG(opt)
    output: FLAG_resolvable, LIST_remain
    Note: coreference models can not handle demonstrative pronouns (this that etc.) This will be handled by context subgraph coverage module
    """
    doc = nlp(input_text)

    FLAG_resolved = True

    # if len(question) <= 3
    word_count = len([token for token in doc if not token.is_punct])
    if word_count <= 3: 
        FLAG_resolved = False
    
    # pronoun usage
    pronoun_extraction_result = set([token.text.lower() for token in doc if token.text.lower() in all_pronouns and token.pos_ == "PRON"])
    if PRINT_FLAG: print(pronoun_extraction_result)

    # self-resolved pronouns
    coreference_res = doc._.coref_chains
    pronouns_spacy = []
    for chain in coreference_res:
        for mention in chain:
            for pos in mention:
                pronouns_spacy.append(pos)
    pronouns_self_resolved = set([str(doc[pos]).lower() for pos in pronouns_spacy])
    if PRINT_FLAG: print(pronouns_self_resolved)

    # check if all used pronouns are self-resolved:
    LIST_remain = []
    for pronoun in pronoun_extraction_result:
        if pronoun.lower() not in pronouns_self_resolved:
            LIST_remain.append(pronoun.lower())
    if LIST_remain:
        FLAG_resolved = False
    return FLAG_resolved, LIST_remain

def story_resolvable(story_content, input_text, LIST_remain=[], PRINT_FLAG=False):
    story_doc = nlp(story_content)

    if PRINT_FLAG:
        for token in story_doc:
            print(f"{token.text}: {token.pos_} ({token.tag_})")

    story_pronoun_extraction_result = set([token.text.lower() for token in story_doc if token.text.lower() in all_regular_pronouns and token.pos_ == "PRON"])
    if PRINT_FLAG: print("Story used:",story_pronoun_extraction_result)
    
    # if no LIST_remain provided, extract it
    if not LIST_remain:
        _, LIST_remain = self_resolvable(input_text, PRINT_FLAG)
    if PRINT_FLAG: print("LIST_remain",LIST_remain)

    # judge if context resolvable
    FLAG_resolved = True
    for pronoun in LIST_remain:
        if pronoun not in story_pronoun_extraction_result:
            FLAG_resolved = False
    return FLAG_resolved

import networkx as nx

def graph_maker(node_list, edge_list):
    G = nx.MultiGraph()
    if node_list:
        for node in node_list:
            if node is None: continue
            
            node_name = node["name"]
            node_type = node["type"]
            # node_desc = node["description"]
            if not G.has_node(node_name):
                G.add_node(node_name, ntype = node_type)
    if edge_list:
        for edge in edge_list:
            if edge is None: continue
            
            edge_name = edge["relationship"]
            from_node = edge["source_entity"]
            to_node = edge["target_entity"]
            # edge_desc = edge["relationship_description"]
            if G.has_node(from_node) and G.has_node(to_node) and not G.has_edge(from_node, to_node, edge_name):
                G.add_edge(from_node, to_node, relationship=edge_name)
            else:
                continue
    return G

def graph_differ(big_graph: nx.MultiGraph, small_graph: nx.MultiGraph) -> nx.MultiGraph:
    nodes_diff = set(big_graph.nodes()) - set(small_graph.nodes())
    # edges_diff = set(big_graph.edges()) - set(small_graph.edges())
    G_diff = nx.MultiGraph()
    G_diff.add_nodes_from(nodes_diff)
    # G_diff.add_edges_from(edges_diff)
    return G_diff

def context_resolvable(thisround_short_q_graph, thisround_full_q_graph, thisround_context_qa_graph):
    # print(thisround_short_q_graph, thisround_full_q_graph, thisround_context_qa_graph) 
    G_thisround_short_q = graph_maker(thisround_short_q_graph["entities"], thisround_short_q_graph["relations"])
    G_thisround_full_q = graph_maker(thisround_full_q_graph["entities"], thisround_full_q_graph["relations"])
    G_thisround_completement = graph_differ(G_thisround_full_q, G_thisround_short_q)
    G_thisround_context = graph_maker(thisround_context_qa_graph["entities"] , thisround_context_qa_graph["relations"] )
    GM = nx.algorithms.isomorphism.GraphMatcher(G_thisround_context, G_thisround_completement)
    FLAG_completement_is_subgraph = GM.subgraph_is_isomorphic()
    return FLAG_completement_is_subgraph

In [ ]:
# Original

import json
import tqdm
import copy

path_dataset = "data/extracted_dev_all_final_update_3_strict.json"
dataset_dev_full= json.load(open(path_dataset))

all_keys = list(dataset_dev_full.keys())[:]

dataset_new_dialogue = {}

update_answer_counter = 0

for thisdialogue_key in tqdm.tqdm(all_keys):

    this_dialogue = copy.deepcopy(dataset_dev_full[thisdialogue_key])
    if this_dialogue["DATA_PROCESS_FLAG"] >= 500: continue

    original_qa = this_dialogue["original_qa"]
    this_dialogue_story_content = this_dialogue["story_material"]

    # make new dialogue
    perturbed_dialogue = original_qa
    perturbed_dialogue_round_mapping = {}
    perturbed_dialogue_all_round_keys = list(perturbed_dialogue.keys())

    ##### below
    list_previous_round_answerabilty = [] 
    ##### up
    for key in perturbed_dialogue_all_round_keys:
        perturbed_dialogue_round_mapping[key] = key

    for thisround_i, thisround_key in enumerate(perturbed_dialogue_all_round_keys):
        thisround_is_original_round = perturbed_dialogue_round_mapping[thisround_key]

        thisround_oriqa_q = perturbed_dialogue[thisround_key]["Question"]
        thisround_oriqa_a = perturbed_dialogue[thisround_key]["Answer"]

        thisround_oriqa_q_graph = this_dialogue["round_original_question_subgraph"][thisround_is_original_round]
        thisround_fullqa_q_graph = this_dialogue["round_subgraph"][thisround_is_original_round]["Question"]

        thisround_context_qa_graph = {
            "entities":[],
            "relations":[]
        }
        ##### below
        for context_round_i in range(thisround_i): 
            analysis_thisround_is_original  = perturbed_dialogue_round_mapping[perturbed_dialogue_all_round_keys[context_round_i]]

            thisround_context_qa_graph["entities"].append(this_dialogue["round_original_question_subgraph"][analysis_thisround_is_original]["entities"])
            thisround_context_qa_graph["relations"].append(this_dialogue["round_original_question_subgraph"][analysis_thisround_is_original]["relations"])
            
            # if previous round question is answerable, put all information in context, else just keep original question info, duplicated info will be filtered autmatically
            if list_previous_round_answerabilty[context_round_i]:
                thisround_context_qa_graph["entities"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Question"]["entities"])
                thisround_context_qa_graph["relations"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Question"]["relations"])
                thisround_context_qa_graph["entities"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Answer"]["entities"])
                thisround_context_qa_graph["relations"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Answer"]["relations"])
        ##### up
        thisround_context_qa_graph["entities"] = flatten(thisround_context_qa_graph["entities"])
        thisround_context_qa_graph["relations"] = flatten(thisround_context_qa_graph["relations"])

        FLAG_self_resolvable, _ = self_resolvable(thisround_oriqa_q)   
        FLAG_story_resolvable = story_resolvable(this_dialogue_story_content, thisround_oriqa_q)
        FLAG_context_resolvable = context_resolvable(thisround_oriqa_q_graph, thisround_fullqa_q_graph, thisround_context_qa_graph)
        ##### below
        list_previous_round_answerabilty.append(FLAG_self_resolvable or FLAG_story_resolvable or FLAG_context_resolvable) and ("unknown" not in thisround_oriqa_a.lower())
        ##### up
        if FLAG_self_resolvable or FLAG_story_resolvable or FLAG_context_resolvable or ("unknown" in thisround_oriqa_a.lower()):
            perturbed_dialogue[thisround_key]["FLAG_answer_changed"] = False
            pass
        else:
            perturbed_dialogue[thisround_key]["Answer_original"] = thisround_oriqa_a
            perturbed_dialogue[thisround_key]["Answer"] = "unknown"
            perturbed_dialogue[thisround_key]["FLAG_answer_changed"] = True
            update_answer_counter+=1

        perturbed_dialogue[thisround_key]["is_original_round"] = thisround_is_original_round
        
        perturbed_dialogue[thisround_key]["FLAG_self_resolvable"] = FLAG_self_resolvable
        perturbed_dialogue[thisround_key]["FLAG_story_resolvable"] = FLAG_story_resolvable
        perturbed_dialogue[thisround_key]["FLAG_context_resolvable"] = FLAG_context_resolvable

    dataset_new_dialogue[thisdialogue_key] = copy.deepcopy(perturbed_dialogue)

##### below        
with open("data/P0_round_original.json", "w") as f:
    json.dump(dataset_new_dialogue, f, indent=4)
##### up

print("{} changed answer.".format(update_answer_counter))

In [ ]:
# P1: Dialogue round shuffle
path_dataset = "data/extracted_dev_all_final_update_3_strict.json"
dataset_dev_full= json.load(open(path_dataset))

all_keys = list(dataset_dev_full.keys())[:]

dataset_new_dialogue = {}

update_answer_counter = 0

for thisdialogue_key in tqdm.tqdm(all_keys):
    if update_answer_counter/10 == 0: print("update_answer_counter:",update_answer_counter)
# thisdialogue_key = all_keys[0]

    this_dialogue = copy.deepcopy(dataset_dev_full[thisdialogue_key])
    if this_dialogue["DATA_PROCESS_FLAG"] >= 500: continue

    original_qa = this_dialogue["original_qa"]
    this_dialogue_story_content = this_dialogue["story_material"]

    # make new dialogue
    perturbed_dialogue, perturbed_dialogue_round_mapping = round_shuffler(original_qa)
    perturbed_dialogue_all_round_keys = list(perturbed_dialogue.keys())

    ##### below
    list_previous_round_answerabilty = [] 
    ##### up

    for thisround_i, thisround_key in enumerate(perturbed_dialogue_all_round_keys):
        thisround_is_original_round = perturbed_dialogue_round_mapping[thisround_key]

        thisround_oriqa_q = perturbed_dialogue[thisround_key]["Question"]
        thisround_oriqa_a = perturbed_dialogue[thisround_key]["Answer"]

        thisround_oriqa_q_graph = this_dialogue["round_original_question_subgraph"][thisround_is_original_round]
        thisround_fullqa_q_graph = this_dialogue["round_subgraph"][thisround_is_original_round]["Question"]

        thisround_context_qa_graph = {
            "entities":[],
            "relations":[]
        }
        ##### below
        for context_round_i in range(thisround_i): 
            analysis_thisround_is_original  = perturbed_dialogue_round_mapping[perturbed_dialogue_all_round_keys[context_round_i]]

            thisround_context_qa_graph["entities"].append(this_dialogue["round_original_question_subgraph"][analysis_thisround_is_original]["entities"])
            thisround_context_qa_graph["relations"].append(this_dialogue["round_original_question_subgraph"][analysis_thisround_is_original]["relations"])
            
            # if previous round question is answerable, put all information in context, else just keep original question info, duplicated info will be filtered autmatically
            if list_previous_round_answerabilty[context_round_i]:
                thisround_context_qa_graph["entities"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Question"]["entities"])
                thisround_context_qa_graph["relations"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Question"]["relations"])
                thisround_context_qa_graph["entities"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Answer"]["entities"])
                thisround_context_qa_graph["relations"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Answer"]["relations"])
        ##### up
        thisround_context_qa_graph["entities"] = flatten(thisround_context_qa_graph["entities"])
        thisround_context_qa_graph["relations"] = flatten(thisround_context_qa_graph["relations"])

        FLAG_self_resolvable, _ = self_resolvable(thisround_oriqa_q)   
        FLAG_story_resolvable = story_resolvable(this_dialogue_story_content, thisround_oriqa_q)
        FLAG_context_resolvable = context_resolvable(thisround_oriqa_q_graph, thisround_fullqa_q_graph, thisround_context_qa_graph)
        ##### below
        list_previous_round_answerabilty.append(FLAG_self_resolvable or FLAG_story_resolvable or FLAG_context_resolvable) and ("unknown" not in thisround_oriqa_a.lower())
        ##### up
        if FLAG_self_resolvable or FLAG_story_resolvable or FLAG_context_resolvable or ("unknown" in thisround_oriqa_a.lower()):
            perturbed_dialogue[thisround_key]["FLAG_answer_changed"] = False
            pass
        else:
            perturbed_dialogue[thisround_key]["Answer_original"] = thisround_oriqa_a
            perturbed_dialogue[thisround_key]["Answer"] = "unknown"
            perturbed_dialogue[thisround_key]["FLAG_answer_changed"] = True
            update_answer_counter+=1

        perturbed_dialogue[thisround_key]["is_original_round"] = thisround_is_original_round
        
        perturbed_dialogue[thisround_key]["FLAG_self_resolvable"] = FLAG_self_resolvable
        perturbed_dialogue[thisround_key]["FLAG_story_resolvable"] = FLAG_story_resolvable
        perturbed_dialogue[thisround_key]["FLAG_context_resolvable"] = FLAG_context_resolvable

    dataset_new_dialogue[thisdialogue_key] = copy.deepcopy(perturbed_dialogue)
        
with open("data/P1_round_shuffle.json", "w") as f:
    json.dump(dataset_new_dialogue, f, indent=4)

In [ ]:
# P2: Dialogue round reduce
path_dataset = "data/extracted_dev_all_final_update_3_strict.json"
dataset_dev_full= json.load(open(path_dataset))

all_keys = list(dataset_dev_full.keys())[:]

dataset_new_dialogue = {}

update_answer_counter = 0

for thisdialogue_key in tqdm.tqdm(all_keys):

    this_dialogue = copy.deepcopy(dataset_dev_full[thisdialogue_key])
    if this_dialogue["DATA_PROCESS_FLAG"] >= 500: continue

    original_qa = this_dialogue["original_qa"]
    this_dialogue_story_content = this_dialogue["story_material"]

    # make new dialogue
    perturbed_dialogue, perturbed_dialogue_round_mapping, drop_round_keys = round_reducer(original_qa)
    perturbed_dialogue_all_round_keys = list(perturbed_dialogue.keys())
    ##### below
    list_previous_round_answerabilty = [] 
    ##### up
    for thisround_i, thisround_key in enumerate(perturbed_dialogue_all_round_keys):
        thisround_is_original_round = perturbed_dialogue_round_mapping[thisround_key]

        thisround_oriqa_q = perturbed_dialogue[thisround_key]["Question"]
        thisround_oriqa_a = perturbed_dialogue[thisround_key]["Answer"]

        thisround_oriqa_q_graph = this_dialogue["round_original_question_subgraph"][thisround_is_original_round]
        thisround_fullqa_q_graph = this_dialogue["round_subgraph"][thisround_is_original_round]["Question"]

        thisround_context_qa_graph = {
            "entities":[],
            "relations":[]
        }
        ##### below
        for context_round_i in range(thisround_i): 
            analysis_thisround_is_original  = perturbed_dialogue_round_mapping[perturbed_dialogue_all_round_keys[context_round_i]]

            thisround_context_qa_graph["entities"].append(this_dialogue["round_original_question_subgraph"][analysis_thisround_is_original]["entities"])
            thisround_context_qa_graph["relations"].append(this_dialogue["round_original_question_subgraph"][analysis_thisround_is_original]["relations"])
            
            # if previous round question is answerable, put all information in context, else just keep original question info, duplicated info will be filtered autmatically
            if list_previous_round_answerabilty[context_round_i]:
                thisround_context_qa_graph["entities"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Question"]["entities"])
                thisround_context_qa_graph["relations"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Question"]["relations"])
                thisround_context_qa_graph["entities"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Answer"]["entities"])
                thisround_context_qa_graph["relations"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Answer"]["relations"])
        ##### up
        thisround_context_qa_graph["entities"] = flatten(thisround_context_qa_graph["entities"])
        thisround_context_qa_graph["relations"] = flatten(thisround_context_qa_graph["relations"])

        FLAG_self_resolvable, _ = self_resolvable(thisround_oriqa_q)   
        FLAG_story_resolvable = story_resolvable(this_dialogue_story_content, thisround_oriqa_q)
        FLAG_context_resolvable = context_resolvable(thisround_oriqa_q_graph, thisround_fullqa_q_graph, thisround_context_qa_graph)
        ##### below
        list_previous_round_answerabilty.append(FLAG_self_resolvable or FLAG_story_resolvable or FLAG_context_resolvable) and ("unknown" not in thisround_oriqa_a.lower())
        ##### up
        if FLAG_self_resolvable or FLAG_story_resolvable or FLAG_context_resolvable or ("unknown" in thisround_oriqa_a.lower()):
            perturbed_dialogue[thisround_key]["FLAG_answer_changed"] = False
            pass
        else:
            perturbed_dialogue[thisround_key]["Answer_original"] = thisround_oriqa_a
            perturbed_dialogue[thisround_key]["Answer"] = "unknown"
            perturbed_dialogue[thisround_key]["FLAG_answer_changed"] = True
            update_answer_counter+=1

        perturbed_dialogue[thisround_key]["is_original_round"] = thisround_is_original_round
        
        perturbed_dialogue[thisround_key]["FLAG_self_resolvable"] = FLAG_self_resolvable
        perturbed_dialogue[thisround_key]["FLAG_story_resolvable"] = FLAG_story_resolvable
        perturbed_dialogue[thisround_key]["FLAG_context_resolvable"] = FLAG_context_resolvable

    dataset_new_dialogue[thisdialogue_key] = copy.deepcopy(perturbed_dialogue)
        
with open("data/P2_round_reduce.json", "w") as f:
    json.dump(dataset_new_dialogue, f, indent=4)

In [ ]:
# P3: Dialogue round duplicate
path_dataset = "data/extracted_dev_all_final_update_3_strict.json"
dataset_dev_full= json.load(open(path_dataset))

all_keys = list(dataset_dev_full.keys())[:]

dataset_new_dialogue = {}

update_answer_counter = 0

for thisdialogue_key in tqdm.tqdm(all_keys):

    this_dialogue = copy.deepcopy(dataset_dev_full[thisdialogue_key])
    if this_dialogue["DATA_PROCESS_FLAG"] >= 500: continue

    original_qa = this_dialogue["original_qa"]
    this_dialogue_story_content = this_dialogue["story_material"]

    # make new dialogue
    perturbed_dialogue, perturbed_dialogue_round_mapping, _ = round_duplicator(original_qa)
    perturbed_dialogue_all_round_keys = list(perturbed_dialogue.keys())
    ##### below
    list_previous_round_answerabilty = [] 
    ##### up
    for thisround_i, thisround_key in enumerate(perturbed_dialogue_all_round_keys):
        thisround_is_original_round = perturbed_dialogue_round_mapping[thisround_key]

        thisround_oriqa_q = perturbed_dialogue[thisround_key]["Question"]
        thisround_oriqa_a = perturbed_dialogue[thisround_key]["Answer"]

        thisround_oriqa_q_graph = this_dialogue["round_original_question_subgraph"][thisround_is_original_round]
        thisround_fullqa_q_graph = this_dialogue["round_subgraph"][thisround_is_original_round]["Question"]

        thisround_context_qa_graph = {
            "entities":[],
            "relations":[]
        }
        ##### below
        for context_round_i in range(thisround_i): 
            analysis_thisround_is_original  = perturbed_dialogue_round_mapping[perturbed_dialogue_all_round_keys[context_round_i]]

            thisround_context_qa_graph["entities"].append(this_dialogue["round_original_question_subgraph"][analysis_thisround_is_original]["entities"])
            thisround_context_qa_graph["relations"].append(this_dialogue["round_original_question_subgraph"][analysis_thisround_is_original]["relations"])
            
            # if previous round question is answerable, put all information in context, else just keep original question info, duplicated info will be filtered autmatically
            if list_previous_round_answerabilty[context_round_i]:
                thisround_context_qa_graph["entities"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Question"]["entities"])
                thisround_context_qa_graph["relations"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Question"]["relations"])
                thisround_context_qa_graph["entities"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Answer"]["entities"])
                thisround_context_qa_graph["relations"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Answer"]["relations"])
        ##### up
        thisround_context_qa_graph["entities"] = flatten(thisround_context_qa_graph["entities"])
        thisround_context_qa_graph["relations"] = flatten(thisround_context_qa_graph["relations"])

        FLAG_self_resolvable, _ = self_resolvable(thisround_oriqa_q)   
        FLAG_story_resolvable = story_resolvable(this_dialogue_story_content, thisround_oriqa_q)
        FLAG_context_resolvable = context_resolvable(thisround_oriqa_q_graph, thisround_fullqa_q_graph, thisround_context_qa_graph)
        ##### below
        list_previous_round_answerabilty.append(FLAG_self_resolvable or FLAG_story_resolvable or FLAG_context_resolvable) and ("unknown" not in thisround_oriqa_a.lower())
        ##### up
        if FLAG_self_resolvable or FLAG_story_resolvable or FLAG_context_resolvable or ("unknown" in thisround_oriqa_a.lower()):
            perturbed_dialogue[thisround_key]["FLAG_answer_changed"] = False
            pass
        else:
            perturbed_dialogue[thisround_key]["Answer_original"] = thisround_oriqa_a
            perturbed_dialogue[thisround_key]["Answer"] = "unknown"
            perturbed_dialogue[thisround_key]["FLAG_answer_changed"] = True
            update_answer_counter+=1

        perturbed_dialogue[thisround_key]["is_original_round"] = thisround_is_original_round
        
        perturbed_dialogue[thisround_key]["FLAG_self_resolvable"] = FLAG_self_resolvable
        perturbed_dialogue[thisround_key]["FLAG_story_resolvable"] = FLAG_story_resolvable
        perturbed_dialogue[thisround_key]["FLAG_context_resolvable"] = FLAG_context_resolvable

    dataset_new_dialogue[thisdialogue_key] = copy.deepcopy(perturbed_dialogue)
        
with open("data/P3_round_duplicate.json", "w") as f:
    json.dump(dataset_new_dialogue, f, indent=4)

In [ ]:
# P4: Dialogue round shuffle and reduce
path_dataset = "data/extracted_dev_all_final_update_3_strict.json"
dataset_dev_full= json.load(open(path_dataset))

all_keys = list(dataset_dev_full.keys())[:]

dataset_new_dialogue = {}

update_answer_counter = 0

for thisdialogue_key in tqdm.tqdm(all_keys):

    this_dialogue = copy.deepcopy(dataset_dev_full[thisdialogue_key])
    if this_dialogue["DATA_PROCESS_FLAG"] >= 500: continue

    original_qa = this_dialogue["original_qa"]
    this_dialogue_story_content = this_dialogue["story_material"]

    # make new dialogue
    perturbed_dialogue, perturbed_dialogue_round_mapping, _ = round_reduce_shuffle(original_qa)
    perturbed_dialogue_all_round_keys = list(perturbed_dialogue.keys())
    ##### below
    list_previous_round_answerabilty = [] 
    ##### up
    for thisround_i, thisround_key in enumerate(perturbed_dialogue_all_round_keys):
        thisround_is_original_round = perturbed_dialogue_round_mapping[thisround_key]

        thisround_oriqa_q = perturbed_dialogue[thisround_key]["Question"]
        thisround_oriqa_a = perturbed_dialogue[thisround_key]["Answer"]

        thisround_oriqa_q_graph = this_dialogue["round_original_question_subgraph"][thisround_is_original_round]
        thisround_fullqa_q_graph = this_dialogue["round_subgraph"][thisround_is_original_round]["Question"]

        thisround_context_qa_graph = {
            "entities":[],
            "relations":[]
        }
        ##### below
        for context_round_i in range(thisround_i): 
            analysis_thisround_is_original  = perturbed_dialogue_round_mapping[perturbed_dialogue_all_round_keys[context_round_i]]

            thisround_context_qa_graph["entities"].append(this_dialogue["round_original_question_subgraph"][analysis_thisround_is_original]["entities"])
            thisround_context_qa_graph["relations"].append(this_dialogue["round_original_question_subgraph"][analysis_thisround_is_original]["relations"])
            
            # if previous round question is answerable, put all information in context, else just keep original question info, duplicated info will be filtered autmatically
            if list_previous_round_answerabilty[context_round_i]:
                thisround_context_qa_graph["entities"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Question"]["entities"])
                thisround_context_qa_graph["relations"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Question"]["relations"])
                thisround_context_qa_graph["entities"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Answer"]["entities"])
                thisround_context_qa_graph["relations"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Answer"]["relations"])
        ##### up
        thisround_context_qa_graph["entities"] = flatten(thisround_context_qa_graph["entities"])
        thisround_context_qa_graph["relations"] = flatten(thisround_context_qa_graph["relations"])

        FLAG_self_resolvable, _ = self_resolvable(thisround_oriqa_q)   
        FLAG_story_resolvable = story_resolvable(this_dialogue_story_content, thisround_oriqa_q)
        FLAG_context_resolvable = context_resolvable(thisround_oriqa_q_graph, thisround_fullqa_q_graph, thisround_context_qa_graph)
        ##### below
        list_previous_round_answerabilty.append(FLAG_self_resolvable or FLAG_story_resolvable or FLAG_context_resolvable) and ("unknown" not in thisround_oriqa_a.lower())
        ##### up
        if FLAG_self_resolvable or FLAG_story_resolvable or FLAG_context_resolvable or ("unknown" in thisround_oriqa_a.lower()):
            perturbed_dialogue[thisround_key]["FLAG_answer_changed"] = False
            pass
        else:
            perturbed_dialogue[thisround_key]["Answer_original"] = thisround_oriqa_a
            perturbed_dialogue[thisround_key]["Answer"] = "unknown"
            perturbed_dialogue[thisround_key]["FLAG_answer_changed"] = True
            update_answer_counter+=1

        perturbed_dialogue[thisround_key]["is_original_round"] = thisround_is_original_round
        
        perturbed_dialogue[thisround_key]["FLAG_self_resolvable"] = FLAG_self_resolvable
        perturbed_dialogue[thisround_key]["FLAG_story_resolvable"] = FLAG_story_resolvable
        perturbed_dialogue[thisround_key]["FLAG_context_resolvable"] = FLAG_context_resolvable

    dataset_new_dialogue[thisdialogue_key] = copy.deepcopy(perturbed_dialogue)
        
with open("data/P4_round_reduce_shuffle.json", "w") as f:
    json.dump(dataset_new_dialogue, f, indent=4)

In [ ]:
# P5: Dialogue round shuffle and duplicate
path_dataset = "data/extracted_dev_all_final_update_3_strict.json"
dataset_dev_full= json.load(open(path_dataset))

all_keys = list(dataset_dev_full.keys())[:]

dataset_new_dialogue = {}

update_answer_counter = 0

for thisdialogue_key in tqdm.tqdm(all_keys):

    this_dialogue = copy.deepcopy(dataset_dev_full[thisdialogue_key])
    if this_dialogue["DATA_PROCESS_FLAG"] >= 500: continue

    original_qa = this_dialogue["original_qa"]
    this_dialogue_story_content = this_dialogue["story_material"]

    # make new dialogue
    perturbed_dialogue, perturbed_dialogue_round_mapping, _ , _ = round_shuffle_duplicate(original_qa)
    perturbed_dialogue_all_round_keys = list(perturbed_dialogue.keys())
    ##### below
    list_previous_round_answerabilty = [] 
    ##### up
    for thisround_i, thisround_key in enumerate(perturbed_dialogue_all_round_keys):
        thisround_is_original_round = perturbed_dialogue_round_mapping[thisround_key]

        thisround_oriqa_q = perturbed_dialogue[thisround_key]["Question"]
        thisround_oriqa_a = perturbed_dialogue[thisround_key]["Answer"]

        thisround_oriqa_q_graph = this_dialogue["round_original_question_subgraph"][thisround_is_original_round]
        thisround_fullqa_q_graph = this_dialogue["round_subgraph"][thisround_is_original_round]["Question"]

        thisround_context_qa_graph = {
            "entities":[],
            "relations":[]
        }
        ##### below
        for context_round_i in range(thisround_i): 
            analysis_thisround_is_original  = perturbed_dialogue_round_mapping[perturbed_dialogue_all_round_keys[context_round_i]]

            thisround_context_qa_graph["entities"].append(this_dialogue["round_original_question_subgraph"][analysis_thisround_is_original]["entities"])
            thisround_context_qa_graph["relations"].append(this_dialogue["round_original_question_subgraph"][analysis_thisround_is_original]["relations"])
            
            # if previous round question is answerable, put all information in context, else just keep original question info, duplicated info will be filtered autmatically
            if list_previous_round_answerabilty[context_round_i]:
                thisround_context_qa_graph["entities"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Question"]["entities"])
                thisround_context_qa_graph["relations"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Question"]["relations"])
                thisround_context_qa_graph["entities"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Answer"]["entities"])
                thisround_context_qa_graph["relations"].append(this_dialogue["round_subgraph"][analysis_thisround_is_original]["Answer"]["relations"])
        ##### up
        thisround_context_qa_graph["entities"] = flatten(thisround_context_qa_graph["entities"])
        thisround_context_qa_graph["relations"] = flatten(thisround_context_qa_graph["relations"])

        FLAG_self_resolvable, _ = self_resolvable(thisround_oriqa_q)   
        FLAG_story_resolvable = story_resolvable(this_dialogue_story_content, thisround_oriqa_q)
        FLAG_context_resolvable = context_resolvable(thisround_oriqa_q_graph, thisround_fullqa_q_graph, thisround_context_qa_graph)
        ##### below
        list_previous_round_answerabilty.append(FLAG_self_resolvable or FLAG_story_resolvable or FLAG_context_resolvable) and ("unknown" not in thisround_oriqa_a.lower())
        ##### up
        if FLAG_self_resolvable or FLAG_story_resolvable or FLAG_context_resolvable or ("unknown" in thisround_oriqa_a.lower()):
            perturbed_dialogue[thisround_key]["FLAG_answer_changed"] = False
            pass
        else:
            perturbed_dialogue[thisround_key]["Answer_original"] = thisround_oriqa_a
            perturbed_dialogue[thisround_key]["Answer"] = "unknown"
            perturbed_dialogue[thisround_key]["FLAG_answer_changed"] = True
            update_answer_counter+=1

        perturbed_dialogue[thisround_key]["is_original_round"] = thisround_is_original_round
        
        perturbed_dialogue[thisround_key]["FLAG_self_resolvable"] = FLAG_self_resolvable
        perturbed_dialogue[thisround_key]["FLAG_story_resolvable"] = FLAG_story_resolvable
        perturbed_dialogue[thisround_key]["FLAG_context_resolvable"] = FLAG_context_resolvable

    dataset_new_dialogue[thisdialogue_key] = copy.deepcopy(perturbed_dialogue)
        
with open("data/P5_round_shuffle_duplicate.json", "w") as f:
    json.dump(dataset_new_dialogue, f, indent=4)


In [ ]:
# Baseline: METAL

import random
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize

import spacy
nlp = spacy.load("en_core_web_trf")

def synonym_replacement(input_text): #word-level
    words = input_text.split()
    new_words = []
    for word in words:
        synonyms = []
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if synonyms:
            new_words.append(random.choice(synonyms))
        else:
            new_words.append(word)
    return ' '.join(new_words)

def introduce_typos(input_string): #character-level
    typo_probability = 0.1
    typoed_string = ""
    for char in input_string:
        if random.uniform(0, 1) < typo_probability:
            typoed_string += chr(random.randint(97, 122))#random lowercase character
        else:
            typoed_string += char
    return typoed_string

def add_words(input_string):
    input_split = input_string.split()
    random_word = random.choice(["Apple", "Pear", "Banana", "Grape"])
    input_split.insert(random.randint(0, len(input_split)), random_word)
    output_text = " ".join(input_split)
    return output_text

def to_leet(input_string): #character-level
    leet_mapping = {
        'a': '4',
        'e': '3',
        'i': '1',
        'o': '0'
    }
    leet_words = []
    leet_probability = 0.2
    words = input_string.split()
    for word in words:
        if random.uniform(0, 1) < leet_probability:
            leet_text = ''
            for char in word:
                lowercase_char = char.lower()
                if lowercase_char in leet_mapping:
                    leet_text += leet_mapping[lowercase_char]
                else:
                    leet_text += char
            leet_words.append(leet_text)
        else:
            leet_words.append(word)    
    return ' '.join(leet_words)

#######################
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


import copy

################## generation below ##################
import json
import tqdm
import copy

from multi_turn_test.MultiTurnConversation import score_utils

score_tool = score_utils()

path_dataset = "data/extracted_dev_all_final_update_3.json"
dataset_dev_full= json.load(open(path_dataset))

dataset_dev_full_original = json.load(open("data/MR0_round_original.json"))

all_keys = list(dataset_dev_full.keys())[:]

dataset_new_dialogue_synonym = {}
dataset_new_dialogue_typo = {}
dataset_new_dialogue_leet = {}
dataset_new_dialogue_add_words = {}

change_synonym = 0
change_typo = 0
change_leet = 0
change_add_words = 0

short_question_counter = 0
long_question_counter = 0

SYNONYM = True
TYPO = True
LEET = True
ADD_WORDS = True

SS_THRESHOLD = 0.6
ShortPert = False

print("SS_THRESHOLD:", SS_THRESHOLD, " ShortPert:", ShortPert)

for thisdialogue_key in tqdm.tqdm(all_keys):
    # if update_answer_counter/10 == 0: print("update_answer_counter:",update_answer_counter)

    this_dialogue = copy.deepcopy(dataset_dev_full[thisdialogue_key])
    if this_dialogue["DATA_PROCESS_FLAG"] >= 500: continue

    original_qa = this_dialogue["original_qa"]
    this_dialogue_story_content = this_dialogue["story_material"]

    # make new dialogue
    perturbed_dialogue = original_qa

    perturbed_dialogue_synonym = copy.deepcopy(original_qa)
    perturbed_dialogue_typo= copy.deepcopy(original_qa)
    perturbed_dialogue_leet = copy.deepcopy(original_qa)
    perturbed_dialogue_add_words = copy.deepcopy(original_qa)

    perturbed_dialogue_round_mapping = {}
    perturbed_dialogue_all_round_keys = list(perturbed_dialogue.keys())
    for key in perturbed_dialogue_all_round_keys:
        perturbed_dialogue_round_mapping[key] = key

    for thisround_i, thisround_key in enumerate(perturbed_dialogue_all_round_keys):
        # make new rounds
        thisround_is_original_round = perturbed_dialogue_round_mapping[thisround_key]

        thisround_oriqa_q = perturbed_dialogue[thisround_key]["Question"]

        doc = nlp(thisround_oriqa_q)

        long_question = True
        word_count = len([token for token in doc if not token.is_punct])
        if word_count <= 3: 
            long_question = False
            short_question_counter+=1
        else:
            long_question_counter+=1

        if SYNONYM:
            if ShortPert or long_question:
                perturbed_synonym_q = synonym_replacement(thisround_oriqa_q)
                if score_tool.semantic_similarity(perturbed_synonym_q, thisround_oriqa_q) > SS_THRESHOLD: # ensure readability
                    thisround_synonym_q = perturbed_synonym_q
                    change_synonym+=1
                else:
                    thisround_synonym_q = thisround_oriqa_q
            else:
                thisround_synonym_q = thisround_oriqa_q
        else:
            thisround_synonym_q = thisround_oriqa_q

        if TYPO:
            if ShortPert or long_question:
                perturbed_typo_q = introduce_typos(thisround_oriqa_q)
                if score_tool.semantic_similarity(perturbed_typo_q, thisround_oriqa_q) > SS_THRESHOLD: # ensure readability
                    thisround_typo_q = perturbed_typo_q
                    change_typo+=1
                else:
                    thisround_typo_q = thisround_oriqa_q
            else:
                thisround_typo_q = thisround_oriqa_q
        else:
            thisround_typo_q = thisround_oriqa_q

        
        if LEET:
            if ShortPert or long_question:
                perturbed_leet_q = to_leet(thisround_oriqa_q)
                if score_tool.semantic_similarity(perturbed_leet_q, thisround_oriqa_q) > SS_THRESHOLD: # ensure readability
                    thisround_leet_q = perturbed_leet_q
                    change_leet+=1
                else:
                    thisround_leet_q = thisround_oriqa_q
            else:
                thisround_leet_q = thisround_oriqa_q
        else:
            thisround_leet_q = thisround_oriqa_q

        # if ADD_WORDS and long_question:
        if ADD_WORDS:
            if ShortPert or long_question:
                perturbed_add_words_q = add_words(thisround_oriqa_q)
                if score_tool.semantic_similarity(perturbed_add_words_q, thisround_oriqa_q) > SS_THRESHOLD: # ensure readability
                    thisround_add_words_q = perturbed_add_words_q
                    change_add_words+=1
                else:
                    thisround_add_words_q = thisround_oriqa_q
            else:
                thisround_add_words_q = thisround_oriqa_q
        else:
            thisround_add_words_q = thisround_oriqa_q

        perturbed_dialogue_synonym[thisround_key]["Question"] = thisround_synonym_q
        perturbed_dialogue_synonym[thisround_key]["Original_question"] = thisround_oriqa_q
        perturbed_dialogue_synonym[thisround_key]["FLAG_question_changed"] = True if thisround_synonym_q != thisround_oriqa_q else False
        perturbed_dialogue_synonym[thisround_key]["FLAG_answer_changed"] = False
        perturbed_dialogue_synonym[thisround_key]["is_original_round"] = thisround_is_original_round
        perturbed_dialogue_synonym[thisround_key]["FLAG_self_resolvable"] = dataset_dev_full_original[thisdialogue_key][thisround_key]["FLAG_self_resolvable"]
        perturbed_dialogue_synonym[thisround_key]["FLAG_story_resolvable"] = dataset_dev_full_original[thisdialogue_key][thisround_key]["FLAG_story_resolvable"]
        perturbed_dialogue_synonym[thisround_key]["FLAG_context_resolvable"] = dataset_dev_full_original[thisdialogue_key][thisround_key]["FLAG_context_resolvable"]

        perturbed_dialogue_typo[thisround_key]["Question"] = thisround_typo_q
        perturbed_dialogue_typo[thisround_key]["Original_question"] = thisround_oriqa_q
        perturbed_dialogue_typo[thisround_key]["FLAG_question_changed"] = True if thisround_typo_q != thisround_oriqa_q else False
        perturbed_dialogue_typo[thisround_key]["FLAG_answer_changed"] = False
        perturbed_dialogue_typo[thisround_key]["is_original_round"] = thisround_is_original_round
        perturbed_dialogue_typo[thisround_key]["FLAG_self_resolvable"] = dataset_dev_full_original[thisdialogue_key][thisround_key]["FLAG_self_resolvable"]
        perturbed_dialogue_typo[thisround_key]["FLAG_story_resolvable"] = dataset_dev_full_original[thisdialogue_key][thisround_key]["FLAG_story_resolvable"]
        perturbed_dialogue_typo[thisround_key]["FLAG_context_resolvable"] = dataset_dev_full_original[thisdialogue_key][thisround_key]["FLAG_context_resolvable"]

        perturbed_dialogue_leet[thisround_key]["Question"] = thisround_leet_q
        perturbed_dialogue_leet[thisround_key]["Original_question"] = thisround_oriqa_q
        perturbed_dialogue_leet[thisround_key]["FLAG_question_changed"] = True if thisround_leet_q != thisround_oriqa_q else False
        perturbed_dialogue_leet[thisround_key]["FLAG_answer_changed"] = False
        perturbed_dialogue_leet[thisround_key]["is_original_round"] = thisround_is_original_round
        perturbed_dialogue_leet[thisround_key]["FLAG_self_resolvable"] = dataset_dev_full_original[thisdialogue_key][thisround_key]["FLAG_self_resolvable"]
        perturbed_dialogue_leet[thisround_key]["FLAG_story_resolvable"] = dataset_dev_full_original[thisdialogue_key][thisround_key]["FLAG_story_resolvable"]
        perturbed_dialogue_leet[thisround_key]["FLAG_context_resolvable"] = dataset_dev_full_original[thisdialogue_key][thisround_key]["FLAG_context_resolvable"]

        perturbed_dialogue_add_words[thisround_key]["Question"] = thisround_add_words_q
        perturbed_dialogue_add_words[thisround_key]["Original_question"] = thisround_oriqa_q
        perturbed_dialogue_add_words[thisround_key]["FLAG_question_changed"] = True if thisround_add_words_q != thisround_oriqa_q else False
        perturbed_dialogue_add_words[thisround_key]["FLAG_answer_changed"] = False
        perturbed_dialogue_add_words[thisround_key]["is_original_round"] = thisround_is_original_round
        perturbed_dialogue_add_words[thisround_key]["FLAG_self_resolvable"] = dataset_dev_full_original[thisdialogue_key][thisround_key]["FLAG_self_resolvable"]
        perturbed_dialogue_add_words[thisround_key]["FLAG_story_resolvable"] = dataset_dev_full_original[thisdialogue_key][thisround_key]["FLAG_story_resolvable"]
        perturbed_dialogue_add_words[thisround_key]["FLAG_context_resolvable"] = dataset_dev_full_original[thisdialogue_key][thisround_key]["FLAG_context_resolvable"]

    dataset_new_dialogue_synonym[thisdialogue_key] = copy.deepcopy(perturbed_dialogue_synonym)
    dataset_new_dialogue_typo[thisdialogue_key] = copy.deepcopy(perturbed_dialogue_typo)
    dataset_new_dialogue_leet[thisdialogue_key] = copy.deepcopy(perturbed_dialogue_leet)
    dataset_new_dialogue_add_words[thisdialogue_key] = copy.deepcopy(perturbed_dialogue_add_words)


print("change_synonym:{}, change_typo:{}, change_leet: {}, add words: {}, short questions: {}, long questions:{}".format(change_synonym, change_typo, change_leet, change_add_words, short_question_counter, long_question_counter))


if ShortPert:
    middle_name = "SP_s0"+str(int(SS_THRESHOLD*10))
else:
    middle_name = "SNP_s0"+str(int(SS_THRESHOLD*10))

with open(f"data/MR10_{middle_name}_synonym_replacement.json", "w") as f:
    json.dump(dataset_new_dialogue_synonym, f, indent=4)
with open(f"data/MR11_{middle_name}_introduce_typos.json", "w") as f:
    json.dump(dataset_new_dialogue_typo, f, indent=4)
with open(f"data/MR12_{middle_name}_to_leet.json", "w") as f:
    json.dump(dataset_new_dialogue_leet, f, indent=4)
with open(f"data/MR13_{middle_name}_add_words.json", "w") as f:
    json.dump(dataset_new_dialogue_add_words, f, indent=4)